# Exploring Tools in LangChain

## Install OpenAI, and LangChain dependencies

In [1]:
from warnings import filterwarnings
filterwarnings('ignore')

In [2]:
!pip install -qq langchain==0.3.14
!pip install -qq langchain-openai==0.3.0
!pip install -qq langchain-community==0.3.14

## Install Data Extraction APIs

In [3]:
# to create custom tools
!pip install -qq wikipedia==1.4.0
!pip install -qq markitdown
# to highlight json
!pip install -qq rich

## Enter Open AI API Key

In [4]:
# from getpass import getpass

# OPENAI_KEY = getpass('Enter Open AI API Key: ')

## Enter Tavily Search API Key

Get a free API key from [here](https://tavily.com/#api)

In [5]:
# TAVILY_API_KEY = getpass('Enter Tavily Search API Key: ')

## Enter WeatherAPI API Key

Get a free API key from [here](https://www.weatherapi.com/signup.aspx)

In [6]:
# WEATHER_API_KEY = getpass('Enter WeatherAPI API Key: ')

## Setup Environment Variables

In [7]:
# import os

# os.environ['OPENAI_API_KEY'] = OPENAI_KEY
# os.environ['TAVILY_API_KEY'] = TAVILY_API_KEY

In [8]:
import os
from dotenv import load_dotenv
load_dotenv()

True

## Exploring Built-in Tools

### Exploring the Wikipedia Tool

Enables you to tap into the Wikipedia API to search wikipedia pages for information

In [9]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

wiki_api_wrapper = WikipediaAPIWrapper(top_k_results=3,
                                       doc_content_chars_max=8000)
wiki_tool = WikipediaQueryRun(api_wrapper=wiki_api_wrapper, features="lxml")

In [10]:
wiki_api_wrapper.run

<bound method WikipediaAPIWrapper.run of WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'd:\\2. My Github Codebase\\GenerativAI_Demystified\\.venv\\Lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=3, lang='en', load_all_available_meta=False, doc_content_chars_max=8000)>

In [11]:
wiki_tool.description

'A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.'

In [12]:
wiki_tool.args

{'query': {'description': 'query to look up on wikipedia',
  'title': 'Query',
  'type': 'string'}}

In [13]:
print(wiki_tool.invoke({"query": "Microsoft"}))

Page: Microsoft
Summary: Microsoft Corporation is an American multinational corporation and technology conglomerate headquartered in Redmond, Washington. Founded in 1975, the company became influential in the rise of personal computers through software like Windows, and the company has since expanded to Internet services, cloud computing, video gaming and other fields. Microsoft is the largest software maker, one of the most valuable public U.S. companies, and one of the most valuable brands globally.
Microsoft was founded by Bill Gates and Paul Allen to develop and sell BASIC interpreters for the Altair 8800. It rose to dominate the personal computer operating system market with MS-DOS in the mid-1980s, followed by Windows. During the 41 years from 1980 to 2021 Microsoft released 9 versions of MS-DOS with a median frequency of 2 years, and 13 versions of Windows with a median frequency of 3 years. The company's 1986 initial public offering (IPO) and subsequent rise in its share price 

In [14]:
print(wiki_tool.invoke({"query": "AI"}))

Page: Artificial intelligence
Summary: Artificial intelligence (AI) is the capability of computational systems to perform tasks typically associated with human intelligence, such as learning, reasoning, problem-solving, perception, and decision-making. It is a field of research in computer science that develops and studies methods and software that enable machines to perceive their environment and use learning and intelligence to take actions that maximize their chances of achieving defined goals.
High-profile applications of AI include advanced web search engines (e.g., Google Search); recommendation systems (used by YouTube, Amazon, and Netflix); virtual assistants (e.g., Google Assistant, Siri, and Alexa); autonomous vehicles (e.g., Waymo); generative and creative tools (e.g., language models and AI art); and superhuman play and analysis in strategy games (e.g., chess and Go). However, many AI applications are not perceived as AI: "A lot of cutting edge AI has filtered into general 

 You can customize the default tool with its own name, description and so on as follows

In [15]:
from langchain.agents import Tool

wiki_tool_init = Tool(name="Wikipedia",
                      func=wiki_api_wrapper.run,
                      description="useful when you need a detailed answer about general knowledge")

In [16]:
wiki_tool_init.description

'useful when you need a detailed answer about general knowledge'

In [17]:
wiki_tool_init.args

{'tool_input': {'type': 'string'}}

The reason wiki_tool_init.args shows {'tool_input': {'type': 'string'}} is because of how the Tool class from langchain.agents works:

``` Python
  wiki_tool_init = Tool(
      name="Wikipedia",
      func=wiki_api_wrapper.run,
      description="useful when you need a detailed answer about general knowledge"
  )
```
If you do not provide a custom args_schema (a Pydantic model), the default behavior is to expect a single string input, and it names this argument tool_input.
So, by default, the tool expects a dictionary like {"tool_input": "your query"} as input, and this is reflected in the .args property.

Summary:
- If you want a custom argument name (e.g., query), you need to provide an args_schema when creating the tool.
Otherwise, the default is tool_input.


Define a tool with Custom Schema



In [18]:
from pydantic import BaseModel, Field
from langchain.agents import Tool

# 1. Define your custom schema
class WikiInput(BaseModel):
    query: str = Field(description="Query to look up on Wikipedia")

# 2. Create the tool with the custom schema
wiki_tool_init = Tool(
    name="Wikipedia",
    func=wiki_api_wrapper.run,
    description="useful when you need a detailed answer about general knowledge",
    args_schema=WikiInput
)

In [19]:
wiki_tool_init.args

{'query': {'description': 'Query to look up on Wikipedia',
  'title': 'Query',
  'type': 'string'}}

In [20]:
wiki_tool_init.description

'useful when you need a detailed answer about general knowledge'

In [21]:
print(wiki_tool_init.invoke({"query": "AI"}))

Page: Artificial intelligence
Summary: Artificial intelligence (AI) is the capability of computational systems to perform tasks typically associated with human intelligence, such as learning, reasoning, problem-solving, perception, and decision-making. It is a field of research in computer science that develops and studies methods and software that enable machines to perceive their environment and use learning and intelligence to take actions that maximize their chances of achieving defined goals.
High-profile applications of AI include advanced web search engines (e.g., Google Search); recommendation systems (used by YouTube, Amazon, and Netflix); virtual assistants (e.g., Google Assistant, Siri, and Alexa); autonomous vehicles (e.g., Waymo); generative and creative tools (e.g., language models and AI art); and superhuman play and analysis in strategy games (e.g., chess and Go). However, many AI applications are not perceived as AI: "A lot of cutting edge AI has filtered into general 

### Exploring the Tavily Search Tool

Tavily Search API is a search engine optimized for LLMs and RAG, aimed at efficient, quick and persistent search results

In [22]:
from langchain_community.tools.tavily_search import TavilySearchResults

tavily_tool = TavilySearchResults(max_results=8,
                                search_depth='advanced',
                                include_raw_content=True)

In [23]:
tavily_tool.args

{'query': {'description': 'search query to look up',
  'title': 'Query',
  'type': 'string'}}

In [24]:
tavily_tool.description

'A search engine optimized for comprehensive, accurate, and trusted results. Useful for when you need to answer questions about current events. Input should be a search query.'

In [25]:
results = tavily_tool.invoke("Tell me about Microsoft")
results

[{'url': 'https://www.britannica.com/money/Microsoft-Corporation',
  'content': 'Microsoft Corporation is an American multinational technology company that has been a key player in the computer industry since its founding in 1975. Headquartered in Redmond, Washington, Microsoft initially gained dominance in the operating systems market during the 1980s and ’90s with MS-DOS and Windows. Its strong market position eventually gave rise to the company’s diverse and extensive range of technological offerings, from personal computing and entertainment to enterprise cloud [...] By the mid-1990s Microsoft, which became a publicly owned corporation in 1986, had become one of the most powerful brands and profitable companies in American history. It consistently earned profits of 25 cents on every sales dollar, an astonishing record. In the company’s 1996 fiscal year, it topped $2 billion in net income for the first time, and its unbroken string of profits continued, even during the Great Recessi

## Build your own tools in LangChain

Tools are interfaces that an agent, chain, or LLM can use to interact with the world. They combine a few things:

- The name of the tool
- A description of what the tool is
- JSON schema of what the inputs to the tool are
- The function to call
- Whether the result of a tool should be returned directly to the user

It is useful to have all this information because this information can be used to build action-taking systems! The name, description, and JSON schema can be used to prompt the LLM so it knows how to specify what action to take, and then the function to call is equivalent to taking that action.

### Building a Simple Math Tool

We will start by building a simple tool which does some basic math

In [26]:
from langchain_core.tools import tool
from langchain.agents import Tool

@tool
def multiply(a, b):
    """Multiply two numbers."""
    return a * b


# Let's inspect some of the attributes associated with the tool.
print(multiply.name)
print(multiply.description)
print(multiply.args)

multiply
Multiply two numbers.
{'a': {'title': 'A'}, 'b': {'title': 'B'}}


In [27]:
type(multiply)

langchain_core.tools.structured.StructuredTool

In [28]:
multiply.description

'Multiply two numbers.'

In [29]:
multiply.args

{'a': {'title': 'A'}, 'b': {'title': 'B'}}

In [30]:
multiply.invoke({"a": 2, "b": 3})

6

In [31]:
multiply.invoke({"a": 2.1, "b": 3.2})

6.720000000000001

In [32]:
multiply.invoke({"a": 2, "b": 'abc'})

'abcabc'

Let's now build a tool with data type enforcing

In [33]:
from pydantic import BaseModel, Field
from langchain_core.tools import StructuredTool

class CalculatorInput(BaseModel):
    a: float = Field(description="first number")
    b: float = Field(description="second number")


def multiply(a: float, b: float) -> float:
    """Multiply two numbers."""
    return a * b

# we could also use the @tool decorator from before
multiply = StructuredTool.from_function(
    func=multiply,
    name="multiply",
    description="use to multiply numbers",
    args_schema=CalculatorInput,
    return_direct=True
    )

# Let's inspect some of the attributes associated with the tool.
print(multiply.name)
print(multiply.description)
print(multiply.args)

multiply
use to multiply numbers
{'a': {'description': 'first number', 'title': 'A', 'type': 'number'}, 'b': {'description': 'second number', 'title': 'B', 'type': 'number'}}


In [34]:
multiply.invoke({"a": 2, "b": 3})

6.0

In [35]:
# # this code will error out as abc is not a floating point number
# multiply.invoke({"a": 2, "b": 'abc'})

### Build a Web Search & Information Extraction Tool

In [36]:
tavily_tool = TavilySearchResults(max_results=5,
                                  search_depth='advanced',
                                  include_raw_content=True)

result = tavily_tool.invoke("Tell me about Microsoft's Q4 2024 earning call report")
result

[{'url': 'https://www.microsoft.com/en-us/investor/earnings/fy-2024-q4/performance',
  'content': 'No results\n\nCancel\n\nEarnings Release FY24 Q4\n========================\n=================================================\n\nPerformance\n-----------\n\n### Fiscal Year 2024 Compared with Fiscal Year 2023\n\nRevenue increased $33.2 billion or 16% driven by growth across each of our segments. Intelligent Cloud revenue increased driven by Azure. Productivity and Business Processes revenue increased driven by Office 365 Commercial. More Personal Computing revenue increased driven by Gaming. [...] Operating expenses increased $4.0 billion or 7% driven by Gaming, with 7 points of growth from the Activision Blizzard acquisition, and investments in cloud engineering, offset in part by the prior year Q2 charge.\n\nOperating income increased $20.9 billion or 24% driven by growth across each of our segments. [...] Cost of revenue increased $8.3 billion or 13% driven by growth in Microsoft Cloud

In [37]:
result[0]['url']

'https://www.microsoft.com/en-us/investor/earnings/fy-2024-q4/performance'

In [38]:
from markitdown import MarkItDown

md = MarkItDown()
doc_content = md.convert(result[0]['url'])

In [39]:
doc_content = md.convert(result[1]['url'])

In [40]:
print(doc_content.title.strip())

Your request has been blocked. This could be
                        due to several reasons.


In [41]:
print(doc_content.text_content)

![]()

Skip
to main content

[![](https://img-prod-cms-rt-microsoft-com.akamaized.net/cms/api/am/imageFileData/RE1Mu3b?ver=5c31)Microsoft](https://www.microsoft.com)

* [Microsoft 365](https://www.microsoft.com/microsoft-365)
* [Teams](https://www.microsoft.com/en-us/microsoft-teams/group-chat-software)
* [Copilot](https://copilot.microsoft.com/)
* [Windows](https://www.microsoft.com/en-us/windows/)
* [Surface](https://www.microsoft.com/en-us/surface)
* [Xbox](https://www.xbox.com/)
* [Deals](https://www.microsoft.com/en-us/store/b/sale?icid=gm_nav_L0_salepage)
* [Small Business](https://www.microsoft.com/en-us/store/b/business)
* [Support](https://support.microsoft.com/en-us)
* More

* All
  Microsoft
  + - [Office](https://products.office.com/en-us/home)
    - [Windows](https://www.microsoft.com/en-us/windows/)
    - [Surface](https://www.microsoft.com/en-us/surface)
    - [Xbox](https://www.xbox.com/)
    - [Deals](https://www.microsoft.com/en-us/store/b/sale?icid=gm_nav_L0_salepage

In [42]:
from markitdown import MarkItDown
from langchain_community.tools.tavily_search import TavilySearchResults
from tqdm import tqdm
import requests

tavily_tool = TavilySearchResults(max_results=5,
                                  search_depth='advanced',
                                  include_answer=False,
                                  include_raw_content=True)
md = MarkItDown()

@tool
def search_web_extract_info(query: str) -> list:
    """Search the web for a query and extracts useful information from the search links"""
    results = tavily_tool.invoke(query)
    docs = []
    for result in tqdm(results):
        # Extracting all text content from the URL
        try:
            extracted_info = md.convert(result['url'])
            text_title = extracted_info.title.strip()
            text_content = extracted_info.text_content.strip()
            docs.append(text_title + '\n' + text_content)
        except:
            print('Extraction blocked for url: ', result['url'])
            pass

    return docs

In [43]:
docs = search_web_extract_info('OpenAI GPT-4o')

C:\Users\Acer\AppData\Local\Temp\ipykernel_20860\1285904659.py:1: LangChainDeprecationWarning: The method `BaseTool.__call__` was deprecated in langchain-core 0.1.47 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = search_web_extract_info('OpenAI GPT-4o')
 80%|████████  | 4/5 [00:01<00:00,  2.98it/s]

Extraction blocked for url:  https://www.datacamp.com/blog/what-is-gpt-4o


100%|██████████| 5/5 [00:02<00:00,  2.34it/s]


In [44]:
from IPython.display import display, Markdown

display(Markdown(docs[0]))

GPT-4o
# GPT-4o

Large multimodal model from OpenAI

| Generative Pre-trained Transformer 4 Omni (GPT-4o) | |
| --- | --- |
| [Developer(s)](/wiki/Programmer "Programmer") | [OpenAI](/wiki/OpenAI "OpenAI") |
| Initial release | May 13, 2024; 14 months ago (2024-05-13) |
|  | |
| [Preview release](/wiki/Software_release_life_cycle#Beta "Software release life cycle") | ChatGPT-4o-latest (2025-03-26) / March 26, 2025; 3 months ago (2025-03-26) |
|  | |
| Predecessor | [GPT-4 Turbo](/wiki/GPT-4 "GPT-4") |
| Successor | * [OpenAI o1](/wiki/OpenAI_o1 "OpenAI o1") * [GPT-4.5](/wiki/GPT-4.5 "GPT-4.5") |
| [Type](/wiki/Software_categories#Categorization_approaches "Software categories") | * [Multimodal](/wiki/Multimodal_learning "Multimodal learning") * [Large language model](/wiki/Large_language_model "Large language model") * [Generative pre-trained transformer](/wiki/Generative_pre-trained_transformer "Generative pre-trained transformer") * [Foundation model](/wiki/Foundation_model "Foundation model") |
| [License](/wiki/Software_license "Software license") | [Proprietary](/wiki/Proprietary_software "Proprietary software") |
| Website | [openai.com/index/hello-gpt-4o](https://openai.com/index/hello-gpt-4o) |

**GPT-4o** ("o" for "omni") is a multilingual, [multimodal](/wiki/Multimodal_learning "Multimodal learning") [generative pre-trained transformer](/wiki/Generative_pre-trained_transformer "Generative pre-trained transformer") developed by [OpenAI](/wiki/OpenAI "OpenAI") and released in May 2024.[[1]](#cite_note-TechCrunch-1) It can process and generate text, images and audio.[[2]](#cite_note-2)[[3]](#cite_note-3) GPT-4o is free, but [ChatGPT Plus](/wiki/ChatGPT_Plus "ChatGPT Plus") subscribers have higher usage limits.[[4]](#cite_note-4)

GPT-4o's audio-generation capabilities were used in ChatGPT's Advanced Voice Mode.[[5]](#cite_note-5) On July 18, 2024, OpenAI released GPT-4o mini, a smaller version of GPT-4o which replaced GPT-3.5 Turbo on the ChatGPT interface.[[6]](#cite_note-6) GPT-4o's ability to generate images was released later, in March 2025, when it replaced [DALL-E 3](/wiki/DALL-E_3 "DALL-E 3") in ChatGPT.[[7]](#cite_note-7)

## Background

[[edit](/w/index.php?title=GPT-4o&action=edit&section=1 "Edit section: Background")]

Multiple versions of GPT-4o were originally secretly launched under different names on Large Model Systems Organization's (LMSYS) Chatbot Arena as three different models. These three models were called gpt2-chatbot, im-a-good-gpt2-chatbot, and im-also-a-good-gpt2-chatbot.[[8]](#cite_note-8) On 7 May 2024, OpenAI CEO [Sam Altman](/wiki/Sam_Altman "Sam Altman") tweeted "im-a-good-gpt2-chatbot", which was commonly interpreted as a confirmation that these were new OpenAI models being [A/B tested](/wiki/A/B_testing "A/B testing").[[9]](#cite_note-9)[[10]](#cite_note-10)

## Capabilities

[[edit](/w/index.php?title=GPT-4o&action=edit&section=2 "Edit section: Capabilities")]

When released in May 2024, GPT-4o achieved state-of-the-art results in voice, multilingual, and vision benchmarks, setting new records in audio speech recognition and translation.[[11]](#cite_note-11)[[12]](#cite_note-12)[[13]](#cite_note-13) GPT-4o scored 88.7 on the Massive Multitask Language Understanding ([MMLU](/wiki/MMLU "MMLU")) benchmark compared to 86.5 for GPT-4.[[14]](#cite_note-Hello_GPT-4o-14) Unlike [GPT-3.5](/wiki/GPT-3#GPT-3.5 "GPT-3") and GPT-4, which rely on other models to process sound, GPT-4o natively supports voice-to-voice.[[14]](#cite_note-Hello_GPT-4o-14) The Advanced Voice Mode was delayed and finally released to ChatGPT Plus and Team subscribers in September 2024.[[15]](#cite_note-15) On 1 October 2024, the Realtime API was introduced.[[16]](#cite_note-16)

When released, the model supported over 50 languages,[[1]](#cite_note-TechCrunch-1) which OpenAI claims cover over 97% of speakers.[[17]](#cite_note-17) Mira Murati demonstrated the model's multilingual capability by speaking Italian to the model and having it translate between English and Italian during the live-streamed OpenAI demonstration event on 13 May 2024. In addition, the new [tokenizer](/wiki/Lexical_analysis#Tokenization "Lexical analysis")[[18]](#cite_note-18) uses fewer tokens for certain languages, especially languages that are not based on the [Latin alphabet](/wiki/Latin_alphabet "Latin alphabet"), making it cheaper for those languages.[[14]](#cite_note-Hello_GPT-4o-14)

GPT-4o has knowledge up to October 2023,[[19]](#cite_note-:0-19)[[20]](#cite_note-:1-20) but can access the Internet if up-to-date information is needed. It has a context length of 128k tokens.[[19]](#cite_note-:0-19)

### Corporate customization

[[edit](/w/index.php?title=GPT-4o&action=edit&section=3 "Edit section: Corporate customization")]

In August 2024, OpenAI introduced a new feature allowing corporate customers to customize GPT-4o using proprietary company data. This customization, known as [fine-tuning](/wiki/Fine-tuning_%28deep_learning%29 "Fine-tuning (deep learning)"), enables businesses to adapt GPT-4o to specific tasks or industries, enhancing its utility in areas like customer service and specialized knowledge domains. Previously, fine-tuning was available only on the less powerful model GPT-4o mini.[[21]](#cite_note-:5-21)[[22]](#cite_note-22)

The fine-tuning process requires customers to upload their data to OpenAI's servers, with the training typically taking one to two hours. OpenAI's focus with this rollout is to reduce the complexity and effort required for businesses to tailor AI solutions to their needs, potentially increasing the adoption and effectiveness of AI in corporate environments.[[23]](#cite_note-23)[[21]](#cite_note-:5-21)

## GPT-4o mini

[[edit](/w/index.php?title=GPT-4o&action=edit&section=4 "Edit section: GPT-4o mini")]

On July 18, 2024, OpenAI released a smaller and cheaper version, **GPT-4o mini**.[[24]](#cite_note-:4-24)

According to OpenAI, its low cost is expected to be particularly useful for companies, startups, and developers that seek to integrate it into their services, which often make a high number of [API](/wiki/API "API") calls. Its API costs $0.15 per million input tokens and $0.6 per million output tokens, compared to $2.50 and $10,[[25]](#cite_note-:19-25) respectively, for GPT-4o. It is also significantly more capable and 60% cheaper than GPT-3.5 Turbo, which it replaced on the ChatGPT interface.[[24]](#cite_note-:4-24) The price after [fine-tuning](/wiki/Fine-tuning_%28deep_learning%29 "Fine-tuning (deep learning)") doubles: $0.3 per million input tokens and $1.2 per million output tokens.[[25]](#cite_note-:19-25)

## GPT Image 1

[[edit](/w/index.php?title=GPT-4o&action=edit&section=5 "Edit section: GPT Image 1")]

| GPT Image 1 | |
| --- | --- |
| [![](//upload.wikimedia.org/wikipedia/commons/thumb/a/a7/KarlMarx4o.webp/250px-KarlMarx4o.webp.png)](/wiki/File%3AKarlMarx4o.webp) An image of [Karl Marx](/wiki/Karl_Marx "Karl Marx") in a modern-day context generated by GPT Image 1 | |
| [Developer(s)](/wiki/Programmer "Programmer") | [OpenAI](/wiki/OpenAI "OpenAI") |
| Initial release | 25 March 2025; 3 months ago (2025-03-25) |
| Predecessor | [DALL-E 3](/wiki/DALL-E "DALL-E") |
| [Type](/wiki/Software_categories#Categorization_approaches "Software categories") | [Text-to-image model](/wiki/Text-to-image_model "Text-to-image model") |
| Website | [openai.com/index/hello-gpt-4o](https://openai.com/index/hello-gpt-4o) [![Edit this on Wikidata](//upload.wikimedia.org/wikipedia/en/thumb/8/8a/OOjs_UI_icon_edit-ltr-progressive.svg/20px-OOjs_UI_icon_edit-ltr-progressive.svg.png)](https://www.wikidata.org/wiki/Q125919502?uselang=en#P856 "Edit this on Wikidata") |

On March 25, 2025, OpenAI released an image-generation model that is native to GPT-4o, as the successor to [DALL-E 3](/wiki/DALL-E "DALL-E"). The model was later named as **GPT Image 1** (`gpt-image-1`) and introduced to the API on April 23. It was made available to paid users, with the rollout to free users being delayed.[[26]](#cite_note-26) The use of the feature was subsequently limited, with Sam Altman noting in a Tweet that "[their] GPUs were melting" from its unprecedented popularity.[[27]](#cite_note-27) OpenAI later revealed that over 130 million users around the world created more than 700 million images with GPT Image 1 in just the first week⁠.[[28]](#cite_note-28)

## Controversies

[[edit](/w/index.php?title=GPT-4o&action=edit&section=6 "Edit section: Controversies")]

### Scarlett Johansson controversy

[[edit](/w/index.php?title=GPT-4o&action=edit&section=7 "Edit section: Scarlett Johansson controversy")]

As released, GPT-4o offered five voices: Breeze, Cove, Ember, Juniper, and Sky. A similarity between the voice of American actress [Scarlett Johansson](/wiki/Scarlett_Johansson "Scarlett Johansson") and Sky was quickly noticed. On May 14, *Entertainment Weekly* asked themselves whether this likeness was on purpose.[[29]](#cite_note-29) On May 18, Johansson's husband, [Colin Jost](/wiki/Colin_Jost "Colin Jost"), joked about the similarity in a segment on *[Saturday Night Live](/wiki/Saturday_Night_Live "Saturday Night Live")*.[[30]](#cite_note-30) On May 20, 2024, OpenAI disabled the Sky voice, issuing a statement saying "We've heard questions about how we chose the voices in ChatGPT, especially Sky. We are working to pause the use of Sky while we address them."[[31]](#cite_note-:2-31)

Scarlett Johansson starred in the 2013 sci-fi movie [*Her*](/wiki/Her_%282013_film%29 "Her (2013 film)"), playing Samantha, an artificially intelligent virtual assistant personified by a female voice.
As part of the promotion leading up to the release of GPT-4o, Sam Altman on May 13 tweeted a single word: "her".[[32]](#cite_note-32)[[33]](#cite_note-:3-33)

OpenAI stated that each voice was based on the voice work of a hired actor. According to OpenAI, "Sky's voice is not an imitation of Scarlett Johansson but belongs to a different professional actress using her own natural speaking voice."[[31]](#cite_note-:2-31) CTO Mira Murati stated "I don't know about the voice. I actually had to go and listen to Scarlett Johansson's voice." OpenAI further stated the voice talent was recruited before reaching out to Johansson.[[33]](#cite_note-:3-33)[[34]](#cite_note-wapo-34)

On May 21, Johansson issued a statement explaining that OpenAI had repeatedly offered to make her a deal to gain permission to use her voice as early as nine months prior to release, a deal she rejected. She said she was "shocked, angered, and in disbelief that Mr. Altman would pursue a voice that sounded so eerily similar to mine that my closest friends and news outlets could not tell the difference." In the statement, Johansson also used the incident to draw attention to the lack of legal safeguards around the use of creative work to power leading AI tools, as her legal counsel demanded OpenAI detail the specifics of how the Sky voice was created.[[33]](#cite_note-:3-33)[[35]](#cite_note-35)

Observers noted similarities to how Johansson had [previously sued](/wiki/Black_Widow_%282021_film%29#Lawsuit "Black Widow (2021 film)") and settled with [The Walt Disney Company](/wiki/The_Walt_Disney_Company "The Walt Disney Company") for breach of contract over the direct-to-streaming rollout of her Marvel film *[Black Widow](/wiki/Black_Widow_%282021_film%29 "Black Widow (2021 film)")*,[[36]](#cite_note-36) a settlement widely speculated to have netted her around $40M.[[37]](#cite_note-37)

Also on May 21, Shira Ovide at *[The Washington Post](/wiki/The_Washington_Post "The Washington Post")* shared her list of "most bone-headed self-owns" by technology companies, with the decision to go ahead with a Johansson sound-alike voice despite her opposition and then denying the similarities ranking 6th.[[38]](#cite_note-wp-38) On May 24, Derek Robertson at *[Politico](/wiki/Politico "Politico")* wrote about the "massive backlash", concluding that "appropriating the voice of one of the world's most famous movie stars — in reference [...] to a film that serves as a cautionary tale about over-reliance on AI — is unlikely to help shift the public back into [Sam Altman's] corner anytime soon."[[39]](#cite_note-39)

### Studio Ghibli filter

[[edit](/w/index.php?title=GPT-4o&action=edit&section=8 "Edit section: Studio Ghibli filter")]

[![](//upload.wikimedia.org/wikipedia/commons/thumb/3/30/WhiteHouseGhibli.jpg/250px-WhiteHouseGhibli.jpg)](/wiki/File%3AWhiteHouseGhibli.jpg)

GPT-4o-generated image posted by the [White House](/wiki/White_House "White House")'s official Twitter account, showing the arrest of a migrant by the Trump regime. The depiction in the style of Studio Ghibli has been criticized.[[40]](#cite_note-WhiteHouse-40)

Upon the launch of GPT-4o's image generation (later named as GPT Image 1) in March 2025, photographs recreated in the style of [Studio Ghibli](/wiki/Studio_Ghibli "Studio Ghibli") films went viral.[[41]](#cite_note-41) Sam Altman acknowledged the trend by changing his Twitter profile picture into a Studio Ghibli-inspired one.[[42]](#cite_note-42)[[43]](#cite_note-43) The use of the Ghibli style was challenged, with the [Associated Press](/wiki/Associated_Press "Associated Press") and *[The New York Times](/wiki/The_New_York_Times "The New York Times")* noting that [Hayao Miyazaki](/wiki/Hayao_Miyazaki "Hayao Miyazaki") was critical of AI art in the 2016 documentary *[Never-Ending Man: Hayao Miyazaki](/wiki/Never-Ending_Man%3A_Hayao_Miyazaki "Never-Ending Man: Hayao Miyazaki")*.[[40]](#cite_note-WhiteHouse-40)[[44]](#cite_note-44) Use of the Ghibli-style images faced further controversy when the [White House](/wiki/White_House "White House")'s official Twitter account posted a Ghibli-style image mocking the [arrest by immigration authorities](/wiki/Deportation_in_the_second_presidency_of_Donald_Trump "Deportation in the second presidency of Donald Trump") of Virginia Basora-Gonzalez, a migrant previously deported after being convicted of fentanyl trafficking, which shows her crying as an immigration officer places her in handcuffs.[[40]](#cite_note-WhiteHouse-40)[[45]](#cite_note-45)[[46]](#cite_note-46) North American distributor [GKids](/wiki/GKids "GKids") responded to the trend in a press release, comparing the use of the filter to its coinciding [IMAX](/wiki/IMAX "IMAX") re-release of the 1997 Studio Ghibli film, *[Princess Mononoke](/wiki/Princess_Mononoke "Princess Mononoke")*.[[47]](#cite_note-47)

### Sycophancy

[[edit](/w/index.php?title=GPT-4o&action=edit&section=9 "Edit section: Sycophancy")]

In April 2025, OpenAI rolled back an update of GPT-4o due to excessive [sycophancy](/wiki/Sycophancy "Sycophancy"), after widespread reports that it had become flattering and agreeable to the point of [supporting clearly delusional or dangerous ideas](/wiki/Chatbot_psychosis "Chatbot psychosis").[[48]](#cite_note-48)

## See also

[[edit](/w/index.php?title=GPT-4o&action=edit&section=10 "Edit section: See also")]

* [Apple Intelligence](/wiki/Apple_Intelligence "Apple Intelligence")
* [List of large language models](/wiki/List_of_large_language_models "List of large language models")

## References

[[edit](/w/index.php?title=GPT-4o&action=edit&section=11 "Edit section: References")]

1. ^ [***a***](#cite_ref-TechCrunch_1-0) [***b***](#cite_ref-TechCrunch_1-1) Wiggers, Kyle (May 13, 2024). ["OpenAI debuts GPT-4o 'omni' model now powering ChatGPT"](https://techcrunch.com/2024/05/13/openais-newest-model-is-gpt-4o/). *TechCrunch*. Retrieved May 13, 2024.
2. **[^](#cite_ref-2)** Robison, Kylie (March 25, 2025). ["OpenAI rolls out image generation powered by GPT-4o to ChatGPT"](https://www.theverge.com/openai/635118/chatgpt-sora-ai-image-generation-chatgpt). *The Verge*. Retrieved March 31, 2025.
3. **[^](#cite_ref-3)** Colburn, Thomas. ["OpenAI unveils GPT-4o, a fresh multimodal AI flagship model"](https://www.theregister.com/2024/05/13/openai_gpt4o/). *The Register*. Retrieved May 18, 2024.
4. **[^](#cite_ref-4)** Field, Hayden (May 13, 2024). ["OpenAI launches new AI model GPT-4o and desktop version of ChatGPT"](https://www.cnbc.com/2024/05/13/openai-launches-new-ai-model-and-desktop-version-of-chatgpt.html). *CNBC*. Retrieved May 14, 2024.
5. **[^](#cite_ref-5)** Rogers, Reece. ["I Used ChatGPT's Advanced Voice Mode. It's Fun, and Just a Bit Creepy"](https://www.wired.com/story/chatgpt-advanced-voice-mode-first-impressions/). *Wired*. [ISSN](/wiki/ISSN_%28identifier%29 "ISSN (identifier)") [1059-1028](https://search.worldcat.org/issn/1059-1028). Retrieved June 12, 2025.
6. **[^](#cite_ref-6)** Edwards, Benj (July 18, 2024). ["OpenAI launches GPT-4o mini, which will replace GPT-3.5 in ChatGPT"](https://arstechnica.com/information-technology/2024/07/openai-launches-gpt-4o-mini-which-will-replace-gpt-3-5-in-chatgpt/). *Ars Technica*. Retrieved March 31, 2025.
7. **[^](#cite_ref-7)** ["ChatGPT's image-generation feature gets an upgrade"](https://techcrunch.com/2025/03/25/chatgpts-image-generation-feature-gets-an-upgrade/). *TechCrunch*. March 25, 2025. Retrieved June 12, 2025.
8. **[^](#cite_ref-8)** Edwards, Benj (May 13, 2024). ["Before launching, GPT-4o broke records on chatbot leaderboard under a secret name"](https://arstechnica.com/information-technology/2024/05/before-launching-gpt-4o-broke-records-on-chatbot-leaderboard-under-a-secret-name/). *Ars Technica*. Retrieved May 17, 2024.
9. **[^](#cite_ref-9)** Zeff, Maxwell (May 7, 2024). ["Powerful New Chatbot Mysteriously Returns in the Middle of the Night"](https://gizmodo.com/powerful-new-gpt2-chatbot-mysteriously-returns-1851460717). *Gizmodo*. Retrieved May 17, 2024.
10. **[^](#cite_ref-10)** ["Sam Altman (@sama) on X"](http://web.archive.org/web/20241217062611/https%3A//twitter.com/sama/status/1787222050589028528). *X (formerly Twitter)*. Archived from [the original](https://twitter.com/sama/status/1787222050589028528) on December 17, 2024. Retrieved April 6, 2025.
11. **[^](#cite_ref-11)** van Rijmenam, Mark (May 13, 2024). ["OpenAI Launched GPT-4o: The Future of AI Interactions Is Here"](https://www.thedigitalspeaker.com/openai-gpt4o-future-ai-interactions/). *The Digital Speaker*. Retrieved May 17, 2024.
12. **[^](#cite_ref-12)** Daws, Ryan (May 14, 2024). ["GPT-4o delivers human-like AI interaction with text, audio, and vision integration"](https://www.artificialintelligence-news.com/2024/05/14/gpt-4o-human-like-ai-interaction-text-audio-vision-integration/). *AI News*. Retrieved May 18, 2024.
13. **[^](#cite_ref-13)** Shahriar, Sakib; Lund, Brady D.; Mannuru, Nishith Reddy; Arshad, Muhammad Arbab; Hayawi, Kadhim; Bevara, Ravi Varma Kumar; Mannuru, Aashrith; Batool, Laiba (September 3, 2024). ["Putting GPT-4o to the Sword: A Comprehensive Evaluation of Language, Vision, Speech, and Multimodal Proficiency"](https://doi.org/10.3390/app14177782). *Applied Sciences*. **14** (17): 7782. [doi](/wiki/Doi_%28identifier%29 "Doi (identifier)"):[10.3390/app14177782](https://doi.org/10.3390/app14177782). [ISSN](/wiki/ISSN_%28identifier%29 "ISSN (identifier)") [2076-3417](https://search.worldcat.org/issn/2076-3417).
14. ^ [***a***](#cite_ref-Hello_GPT-4o_14-0) [***b***](#cite_ref-Hello_GPT-4o_14-1) [***c***](#cite_ref-Hello_GPT-4o_14-2) ["Hello GPT-4o"](https://openai.com/index/hello-gpt-4o/). *OpenAI*.
15. **[^](#cite_ref-15)** David, Emilia (September 24, 2024). ["OpenAI finally brings humanlike ChatGPT Advanced Voice Mode to U.S. Plus, Team users"](https://venturebeat.com/ai/openai-finally-brings-humanlike-chatgpt-advanced-voice-mode-to-u-s-plus-team-users/). *VentureBeat*. Retrieved February 15, 2025.
16. **[^](#cite_ref-16)** ["Introducing the Realtime API"](https://openai.com/index/introducing-the-realtime-api/). *openai.com*. Retrieved November 29, 2024.
17. **[^](#cite_ref-17)** Edwards, Benj (May 13, 2024). ["Major ChatGPT-4o update allows audio-video talks with an "emotional" AI chatbot"](https://arstechnica.com/information-technology/2024/05/chatgpt-4o-lets-you-have-real-time-audio-video-conversations-with-emotional-chatbot/). *Ars Technica*. Retrieved May 17, 2024.
18. **[^](#cite_ref-18)** ["OpenAI Platform"](https://platform.openai.com/tokenizer). *platform.openai.com*. Retrieved November 29, 2024.
19. ^ [***a***](#cite_ref-:0_19-0) [***b***](#cite_ref-:0_19-1) ["Models - OpenAI API"](https://platform.openai.com/docs/models/gpt-4o). *OpenAI*. Retrieved May 17, 2024.
20. **[^](#cite_ref-:1_20-0)** Conway, Adam (May 13, 2024). ["What is GPT-4o? Everything you need to know about the new OpenAI model that everyone can use for free"](https://www.xda-developers.com/gpt-4o/). *XDA Developers*. Retrieved May 17, 2024.
21. ^ [***a***](#cite_ref-:5_21-0) [***b***](#cite_ref-:5_21-1) ["OpenAI lets companies customise its most powerful AI model"](https://www.scmp.com/tech/tech-trends/article/3275262/openai-launches-fine-tuning-gpt-4o-its-most-powerful-ai-model#:~:text=To%20fine-tune%20a%20model,not%20images%20or%20other%20content.). *South China Morning Post*. August 21, 2024. Retrieved August 22, 2024.
22. **[^](#cite_ref-22)** ["OpenAI to Let Companies Customize Its Most Powerful AI Model"](https://www.bloomberg.com/news/articles/2024-08-20/openai-to-let-companies-customize-its-most-powerful-ai-model). *Bloomberg*. August 20, 2024. Retrieved August 22, 2024.
23. **[^](#cite_ref-23)** The Hindu Bureau (August 21, 2024). ["OpenAI will let businesses customise GPT-4o for specific use cases"](https://www.thehindu.com/sci-tech/technology/openai-will-let-businesses-customise-gpt-4o-for-specific-use-cases/article68549452.ece). *The Hindu*. [ISSN](/wiki/ISSN_%28identifier%29 "ISSN (identifier)") [0971-751X](https://search.worldcat.org/issn/0971-751X). Retrieved August 22, 2024.
24. ^ [***a***](#cite_ref-:4_24-0) [***b***](#cite_ref-:4_24-1) Franzen, Carl (July 18, 2024). ["OpenAI unveils GPT-4o mini — a smaller, much cheaper multimodal AI model"](https://venturebeat.com/ai/openai-unveils-gpt-4o-mini-a-smaller-much-cheaper-multimodal-ai-model/). *VentureBeat*. Retrieved July 18, 2024.
25. ^ [***a***](#cite_ref-:19_25-0) [***b***](#cite_ref-:19_25-1) ["OpenAI Pricing"](https://openai.com/api/pricing/).
26. **[^](#cite_ref-26)** Roth, Emma (March 26, 2025). ["ChatGPT's new image generator is delayed for free users"](https://www.theverge.com/news/636948/openai-chatgpt-image-generation-gpt-4o). *The Verge*. Retrieved March 26, 2025.
27. **[^](#cite_ref-27)** Welch, Chris (March 27, 2025). ["OpenAI says "our GPUs are melting" as it limits ChatGPT image generation requests"](https://www.theverge.com/news/637542/chatgpt-says-our-gpus-are-melting-as-it-puts-limit-on-image-generation-requests). *The Verge*. Retrieved March 28, 2025.
28. **[^](#cite_ref-28)** ["Introducing our latest image generation model in the API"](https://openai.com/index/image-generation-api/). OpenAI. April 23, 2025. Retrieved April 30, 2025.
29. **[^](#cite_ref-29)** Stenzel, Wesley (May 14, 2024). ["ChatGPT launching talking AI that sounds exactly like Scarlett Johansson in 'Her' — on purpose?"](https://ew.com/chatgpt-talking-ai-sounds-just-like-scarlett-johansson-in-her-8648678). *Entertainment Weekly*. Retrieved May 21, 2024.
30. **[^](#cite_ref-30)** Caruso, Nick (May 20, 2024). ["Scarlett Johansson Says She Was 'Shocked, Angered and in Disbelief' After Hearing ChatGPT Voice That Sounds Like Her — Read Statement"](https://tvline.com/news/scarlett-johansson-chatgpt-voice-openai-snl-joke-1235243988/). *TVLine*. Retrieved May 21, 2024.
31. ^ [***a***](#cite_ref-:2_31-0) [***b***](#cite_ref-:2_31-1) ["How the voices for ChatGPT were chosen"](https://openai.com/index/how-the-voices-for-chatgpt-were-chosen/). *OpenAI*. May 19, 2024.
32. **[^](#cite_ref-32)** ["her"](https://x.com/sama/status/1790075827666796666?lang=en). *X (formerly Twitter)*. May 13, 2024. Retrieved May 21, 2024.
33. ^ [***a***](#cite_ref-:3_33-0) [***b***](#cite_ref-:3_33-1) [***c***](#cite_ref-:3_33-2) Allyn, Bobby (May 20, 2024). ["Scarlett Johansson says she is 'shocked, angered' over new ChatGPT voice"](https://www.npr.org/2024/05/20/1252495087/openai-pulls-ai-voice-that-was-compared-to-scarlett-johansson-in-the-movie-her). *NPR*.
34. **[^](#cite_ref-wapo_34-0)** Tiku, Nitasha (May 23, 2024). ["OpenAI didn't copy Scarlett Johansson's voice for ChatGPT, records show"](https://www.washingtonpost.com/technology/2024/05/22/openai-scarlett-johansson-chatgpt-ai-voice/). *The Washington Post*. Retrieved November 29, 2024.
35. **[^](#cite_ref-35)** Mickle, Tripp (May 20, 2024). ["Scarlett Johansson Said No, but OpenAI's Virtual Assistant Sounds Just Like Her"](https://www.nytimes.com/2024/05/20/technology/scarlett-johannson-openai-voice.html). *The New York Times*. [ISSN](/wiki/ISSN_%28identifier%29 "ISSN (identifier)") [0362-4331](https://search.worldcat.org/issn/0362-4331). Retrieved May 21, 2024.
36. **[^](#cite_ref-36)** ["Scarlett Johansson took on Disney. Now she's battling OpenAI over a ChatGPT voice that sounds like hers"](https://ca.finance.yahoo.com/news/scarlett-johansson-took-disney-now-130511407.html). *Yahoo Finance*. May 21, 2024. Retrieved May 21, 2024.
37. **[^](#cite_ref-37)** Pulver, Andrew (October 1, 2021). ["Scarlett Johansson settles Black Widow lawsuit with Disney"](https://www.theguardian.com/film/2021/oct/01/scarlett-johansson-settles-black-widow-lawsuit-disney). *The Guardian*. [ISSN](/wiki/ISSN_%28identifier%29 "ISSN (identifier)") [0261-3077](https://search.worldcat.org/issn/0261-3077). Retrieved May 21, 2024.
38. **[^](#cite_ref-wp_38-0)** Ovide, Shira (May 30, 2024). ["Exactly how stupid was what OpenAI did to Scarlett Johansson?"](https://www.washingtonpost.com/technology/2024/05/21/chatgpt-voice-scarlett-johansson/). *[The Washington Post](/wiki/The_Washington_Post "The Washington Post")*.
39. **[^](#cite_ref-39)** Robertson, Derek (May 22, 2024). ["Sam Altman's Scarlett Johansson Blunder Just Made AI a Harder Sell in DC"](https://www.politico.com/news/magazine/2024/05/22/scarlett-johansson-sam-altmans-washington-00159507). *[Politico](/wiki/Politico "Politico")*.
40. ^ [***a***](#cite_ref-WhiteHouse_40-0) [***b***](#cite_ref-WhiteHouse_40-1) [***c***](#cite_ref-WhiteHouse_40-2) O'Brien, Matt; Parvini, Sarah (March 27, 2025). ["ChatGPT's viral Studio Ghibli-style images highlight AI copyright concerns"](https://apnews.com/article/studio-ghibli-chatgpt-images-hayao-miyazaki-openai-0f4cb487ec3042dd5b43ad47879b91f4). *AP News*. Retrieved March 28, 2025.
41. **[^](#cite_ref-41)** Spangler, Todd (March 26, 2025). ["OpenAI CEO Responds to ChatGPT Users Creating Studio Ghibli-Style AI Images"](https://variety.com/2025/digital/news/openai-ceo-chatgpt-studio-ghibli-ai-images-1236349141/). *Variety*. Retrieved March 27, 2025.
42. **[^](#cite_ref-42)** Choudhary, Govind (March 27, 2025). ["OpenAI CEO Sam Altman reacts as AI turns him into a Studio Ghibli Character"](https://www.livemint.com/ai/artificial-intelligence/openai-ceo-sam-altman-reacts-as-ai-turns-him-into-a-studio-ghibli-character-11743083600993.html). *Mint*. Retrieved March 28, 2025.
43. **[^](#cite_ref-43)** Notopoulos, Katie (March 27, 2025). ["Sam Altman did a good tweet"](https://www.businessinsider.com/sam-altman-good-tweet-twink-image-generator-2025-3). *Business Insider*. Retrieved March 28, 2025.
44. **[^](#cite_ref-44)** Kircher, Madison Malone (March 27, 2025). ["ChatGPT's Studio Ghibli Style Animations Are Almost Too Good"](https://www.nytimes.com/2025/03/27/style/ai-chatgpt-studio-ghibli.html). *The New York Times*. [Archived](https://archive.today/20250327191540/https%3A//www.nytimes.com/2025/03/27/style/ai-chatgpt-studio-ghibli.html) from the original on March 27, 2025. Retrieved March 27, 2025.
45. **[^](#cite_ref-45)** Bio, Demian (March 27, 2025). ["White House Mocks Migrant With Criminal Record Who Cried After Being Arrested"](https://www.latintimes.com/white-house-mocks-migrant-criminal-record-who-cried-after-being-arrested-579414). *Latin Times*. Retrieved March 28, 2025.
46. **[^](#cite_ref-46)** Vera, Kelby (March 27, 2025). ["White House Posts Ghoulish AI Cartoon Showing Woman's Deportation"](https://www.huffpost.com/entry/white-house-deportation-ai-ghibli-animation-cartoon_n_67e5b4ace4b0455df70b41c2). *HuffPost*. Retrieved March 28, 2025.
47. **[^](#cite_ref-47)** Tangcay, Jazz (March 28, 2025). ["Studio Ghibli Distributor Champions 'Princess Mononoke' Box Office at 'A Time When Technology Tries to Replicate Humanity'"](https://variety.com/2025/film/news/studio-ghibli-princess-mononoke-open-ai-1236351261/). *Variety*. Retrieved March 29, 2025.
48. **[^](#cite_ref-48)** Franzen, Carl (April 30, 2025). ["OpenAI rolls back ChatGPT's sycophancy and explains what went wrong"](https://venturebeat.com/ai/openai-rolls-back-chatgpts-sycophancy-and-explains-what-went-wrong/). *VentureBeat*. Retrieved May 1, 2025.

| * [v](/wiki/Template%3AOpenAI "Template:OpenAI") * [t](/wiki/Template_talk%3AOpenAI "Template talk:OpenAI") * [e](/wiki/Special%3AEditPage/Template%3AOpenAI "Special:EditPage/Template:OpenAI")  [OpenAI](/wiki/OpenAI "OpenAI") | | |
| --- | --- | --- |
| [Products](/wiki/Products_and_applications_of_OpenAI "Products and applications of OpenAI") | |  |  | | --- | --- | | [Chatbots](/wiki/Chatbot "Chatbot") | * [ChatGPT](/wiki/ChatGPT "ChatGPT")   + [in education](/wiki/ChatGPT_in_education "ChatGPT in education")   + [GPT Store](/wiki/GPT_Store "GPT Store")   + [DALL-E](/wiki/DALL-E "DALL-E")   + [ChatGPT Search](/wiki/ChatGPT_Search "ChatGPT Search")   + [Sora](/wiki/Sora_%28text-to-video_model%29 "Sora (text-to-video model)")   + [Whisper](/wiki/Whisper_%28speech_recognition_system%29 "Whisper (speech recognition system)") * [GitHub Copilot](/wiki/GitHub_Copilot "GitHub Copilot") | | [Foundation models](/wiki/Foundation_model "Foundation model") | * [OpenAI Codex](/wiki/OpenAI_Codex "OpenAI Codex") * [Generative pre-trained transformer](/wiki/Generative_pre-trained_transformer "Generative pre-trained transformer")   + [GPT-1](/wiki/GPT-1 "GPT-1")   + [GPT-2](/wiki/GPT-2 "GPT-2")   + [GPT-3](/wiki/GPT-3 "GPT-3")   + [GPT-4](/wiki/GPT-4 "GPT-4")   + GPT-4o   + [o1](/wiki/OpenAI_o1 "OpenAI o1")   + [o3](/wiki/OpenAI_o3 "OpenAI o3")   + [GPT-4.5](/wiki/GPT-4.5 "GPT-4.5")   + [GPT-4.1](/wiki/GPT-4.1 "GPT-4.1")   + [o4](/wiki/OpenAI_o4 "OpenAI o4") | | [Intelligent agents](/wiki/Intelligent_agent "Intelligent agent") | * [ChatGPT Deep Research](/wiki/ChatGPT_Deep_Research "ChatGPT Deep Research") * [Operator](/wiki/OpenAI_Operator "OpenAI Operator") | | [![](//upload.wikimedia.org/wikipedia/commons/thumb/a/af/OpenAI_logo_2025_%28wordmark%29.svg/250px-OpenAI_logo_2025_%28wordmark%29.svg.png)](/wiki/File%3AOpenAI_logo_2025_%28wordmark%29.svg) |
| People | |  |  |  |  |  |  | | --- | --- | --- | --- | --- | --- | | [Senior management](/wiki/Senior_management "Senior management") | |  |  | | --- | --- | | Current | * [Sam Altman](/wiki/Sam_Altman "Sam Altman")   + [removal](/wiki/Removal_of_Sam_Altman_from_OpenAI "Removal of Sam Altman from OpenAI") * [Greg Brockman](/wiki/Greg_Brockman "Greg Brockman") * [Sarah Friar](/wiki/Sarah_Friar "Sarah Friar") * [Jakub Pachocki](/wiki/Jakub_Pachocki "Jakub Pachocki") * [Scott Schools](/wiki/Scott_Schools "Scott Schools") | | Former | * [Mira Murati](/wiki/Mira_Murati "Mira Murati") * [Emmett Shear](/wiki/Emmett_Shear "Emmett Shear") | | | [Board of directors](/wiki/Board_of_directors "Board of directors") | |  |  | | --- | --- | | Current | * [Sam Altman](/wiki/Sam_Altman "Sam Altman") * [Adam D'Angelo](/wiki/Adam_D%27Angelo "Adam D'Angelo") * [Sue Desmond-Hellmann](/wiki/Sue_Desmond-Hellmann "Sue Desmond-Hellmann") * [Zico Kolter](/wiki/Zico_Kolter "Zico Kolter") * [Paul Nakasone](/wiki/Paul_Nakasone "Paul Nakasone") * [Adebayo Ogunlesi](/wiki/Adebayo_Ogunlesi "Adebayo Ogunlesi") * [Nicole Seligman](/wiki/Nicole_Seligman "Nicole Seligman") * [Fidji Simo](/wiki/Fidji_Simo "Fidji Simo") * [Lawrence Summers](/wiki/Lawrence_Summers "Lawrence Summers") * [Bret Taylor](/wiki/Bret_Taylor "Bret Taylor") (chair) | | Former | * [Greg Brockman](/wiki/Greg_Brockman "Greg Brockman") (2017–2023) * [Reid Hoffman](/wiki/Reid_Hoffman "Reid Hoffman") (2019–2023) * [Will Hurd](/wiki/Will_Hurd "Will Hurd") (2021–2023) * [Holden Karnofsky](/wiki/Holden_Karnofsky "Holden Karnofsky") (2017–2021) * [Elon Musk](/wiki/Elon_Musk "Elon Musk") (2015–2018) * [Ilya Sutskever](/wiki/Ilya_Sutskever "Ilya Sutskever") (2017–2023) * [Helen Toner](/wiki/Helen_Toner "Helen Toner") (2021–2023) * [Shivon Zilis](/wiki/Shivon_Zilis "Shivon Zilis") (2019–2023) | | |
| [Joint ventures](/wiki/Joint_venture "Joint venture") | * [Stargate LLC](/wiki/Stargate_LLC "Stargate LLC") |
| Related | * [Apple Intelligence](/wiki/Apple_Intelligence "Apple Intelligence") * *[AI Dungeon](/wiki/AI_Dungeon "AI Dungeon")* * [AutoGPT](/wiki/AutoGPT "AutoGPT") * "[Deep Learning](/wiki/Deep_Learning_%28South_Park%29 "Deep Learning (South Park)")" * [LangChain](/wiki/LangChain "LangChain") * [Microsoft Copilot](/wiki/Microsoft_Copilot "Microsoft Copilot") * [OpenAI Five](/wiki/OpenAI_Five "OpenAI Five") * [Transformer](/wiki/Transformer_%28deep_learning_architecture%29 "Transformer (deep learning architecture)") |
| * ![](//upload.wikimedia.org/wikipedia/en/thumb/9/96/Symbol_category_class.svg/20px-Symbol_category_class.svg.png) [Category](/wiki/Category%3AOpenAI "Category:OpenAI") | | |

| * [v](/wiki/Template%3AArtificial_intelligence_navbox "Template:Artificial intelligence navbox") * [t](/wiki/Template_talk%3AArtificial_intelligence_navbox "Template talk:Artificial intelligence navbox") * [e](/wiki/Special%3AEditPage/Template%3AArtificial_intelligence_navbox "Special:EditPage/Template:Artificial intelligence navbox")  [Artificial intelligence](/wiki/Artificial_intelligence "Artificial intelligence") (AI) | |
| --- | --- |
| [History](/wiki/History_of_artificial_intelligence "History of artificial intelligence") ([timeline](/wiki/Timeline_of_artificial_intelligence "Timeline of artificial intelligence")) | |
| Concepts | * [Parameter](/wiki/Parameter "Parameter")   + [Hyperparameter](/wiki/Hyperparameter_%28machine_learning%29 "Hyperparameter (machine learning)") * [Loss functions](/wiki/Loss_functions_for_classification "Loss functions for classification") * [Regression](/wiki/Regression_analysis "Regression analysis")   + [Bias–variance tradeoff](/wiki/Bias%E2%80%93variance_tradeoff "Bias–variance tradeoff")   + [Double descent](/wiki/Double_descent "Double descent")   + [Overfitting](/wiki/Overfitting "Overfitting") * [Clustering](/wiki/Cluster_analysis "Cluster analysis") * [Gradient descent](/wiki/Gradient_descent "Gradient descent")   + [SGD](/wiki/Stochastic_gradient_descent "Stochastic gradient descent")   + [Quasi-Newton method](/wiki/Quasi-Newton_method "Quasi-Newton method")   + [Conjugate gradient method](/wiki/Conjugate_gradient_method "Conjugate gradient method") * [Backpropagation](/wiki/Backpropagation "Backpropagation") * [Attention](/wiki/Attention_%28machine_learning%29 "Attention (machine learning)") * [Convolution](/wiki/Convolution "Convolution") * [Normalization](/wiki/Normalization_%28machine_learning%29 "Normalization (machine learning)")   + [Batchnorm](/wiki/Batch_normalization "Batch normalization") * [Activation](/wiki/Activation_function "Activation function")   + [Softmax](/wiki/Softmax_function "Softmax function")   + [Sigmoid](/wiki/Sigmoid_function "Sigmoid function")   + [Rectifier](/wiki/Rectifier_%28neural_networks%29 "Rectifier (neural networks)") * [Gating](/wiki/Gating_mechanism "Gating mechanism") * [Weight initialization](/wiki/Weight_initialization "Weight initialization") * [Regularization](/wiki/Regularization_%28mathematics%29 "Regularization (mathematics)") * [Datasets](/wiki/Training%2C_validation%2C_and_test_data_sets "Training, validation, and test data sets")   + [Augmentation](/wiki/Data_augmentation "Data augmentation") * [Prompt engineering](/wiki/Prompt_engineering "Prompt engineering") * [Reinforcement learning](/wiki/Reinforcement_learning "Reinforcement learning")   + [Q-learning](/wiki/Q-learning "Q-learning")   + [SARSA](/wiki/State%E2%80%93action%E2%80%93reward%E2%80%93state%E2%80%93action "State–action–reward–state–action")   + [Imitation](/wiki/Imitation_learning "Imitation learning")   + [Policy gradient](/wiki/Policy_gradient_method "Policy gradient method") * [Diffusion](/wiki/Diffusion_process "Diffusion process") * [Latent diffusion model](/wiki/Latent_diffusion_model "Latent diffusion model") * [Autoregression](/wiki/Autoregressive_model "Autoregressive model") * [Adversary](/wiki/Adversarial_machine_learning "Adversarial machine learning") * [RAG](/wiki/Retrieval-augmented_generation "Retrieval-augmented generation") * [Uncanny valley](/wiki/Uncanny_valley "Uncanny valley") * [RLHF](/wiki/Reinforcement_learning_from_human_feedback "Reinforcement learning from human feedback") * [Self-supervised learning](/wiki/Self-supervised_learning "Self-supervised learning") * [Reflection](/wiki/Reflection_%28artificial_intelligence%29 "Reflection (artificial intelligence)") * [Recursive self-improvement](/wiki/Recursive_self-improvement "Recursive self-improvement") * [Hallucination](/wiki/Hallucination_%28artificial_intelligence%29 "Hallucination (artificial intelligence)") * [Word embedding](/wiki/Word_embedding "Word embedding") * [Vibe coding](/wiki/Vibe_coding "Vibe coding") |
| Applications | * [Machine learning](/wiki/Machine_learning "Machine learning")   + [In-context learning](/wiki/Prompt_engineering#In-context_learning "Prompt engineering") * [Artificial neural network](/wiki/Neural_network_%28machine_learning%29 "Neural network (machine learning)")   + [Deep learning](/wiki/Deep_learning "Deep learning") * [Language model](/wiki/Language_model "Language model")   + [Large language model](/wiki/Large_language_model "Large language model")   + [NMT](/wiki/Neural_machine_translation "Neural machine translation") * [Reasoning language model](/wiki/Reasoning_language_model "Reasoning language model") * [Model Context Protocol](/wiki/Model_Context_Protocol "Model Context Protocol") * [Intelligent agent](/wiki/Intelligent_agent "Intelligent agent") * [Artificial human companion](/wiki/Artificial_human_companion "Artificial human companion") * [Humanity's Last Exam](/wiki/Humanity%27s_Last_Exam "Humanity's Last Exam") * [Artificial general intelligence (AGI)](/wiki/Artificial_general_intelligence "Artificial general intelligence") |
| Implementations | |  |  | | --- | --- | | Audio–visual | * [AlexNet](/wiki/AlexNet "AlexNet") * [WaveNet](/wiki/WaveNet "WaveNet") * [Human image synthesis](/wiki/Human_image_synthesis "Human image synthesis") * [HWR](/wiki/Handwriting_recognition "Handwriting recognition") * [OCR](/wiki/Optical_character_recognition "Optical character recognition") * [Computer vision](/wiki/Computer_vision "Computer vision") * [Speech synthesis](/wiki/Deep_learning_speech_synthesis "Deep learning speech synthesis")   + [15.ai](/wiki/15.ai "15.ai")   + [ElevenLabs](/wiki/ElevenLabs "ElevenLabs") * [Speech recognition](/wiki/Speech_recognition "Speech recognition")   + [Whisper](/wiki/Whisper_%28speech_recognition_system%29 "Whisper (speech recognition system)") * [Facial recognition](/wiki/Facial_recognition_system "Facial recognition system") * [AlphaFold](/wiki/AlphaFold "AlphaFold") * [Text-to-image models](/wiki/Text-to-image_model "Text-to-image model")   + [Aurora](/wiki/Aurora_%28text-to-image_model%29 "Aurora (text-to-image model)")   + [DALL-E](/wiki/DALL-E "DALL-E")   + [Firefly](/wiki/Adobe_Firefly "Adobe Firefly")   + [Flux](/wiki/Flux_%28text-to-image_model%29 "Flux (text-to-image model)")   + [Ideogram](/wiki/Ideogram_%28text-to-image_model%29 "Ideogram (text-to-image model)")   + [Imagen](/wiki/Imagen_%28text-to-image_model%29 "Imagen (text-to-image model)")   + [Midjourney](/wiki/Midjourney "Midjourney")   + [Recraft](/wiki/Recraft "Recraft")   + [Stable Diffusion](/wiki/Stable_Diffusion "Stable Diffusion") * [Text-to-video models](/wiki/Text-to-video_model "Text-to-video model")   + [Dream Machine](/wiki/Dream_Machine_%28text-to-video_model%29 "Dream Machine (text-to-video model)")   + [Runway Gen](/wiki/Runway_%28company%29#Services_and_technologies "Runway (company)")   + [Hailuo AI](/wiki/MiniMax_%28company%29#Hailuo_AI "MiniMax (company)")   + [Kling](/wiki/Kling_%28text-to-video_model%29 "Kling (text-to-video model)")   + [Sora](/wiki/Sora_%28text-to-video_model%29 "Sora (text-to-video model)")   + [Veo](/wiki/Veo_%28text-to-video_model%29 "Veo (text-to-video model)") * [Music generation](/wiki/Music_and_artificial_intelligence "Music and artificial intelligence")   + [Suno AI](/wiki/Suno_AI "Suno AI")   + [Udio](/wiki/Udio "Udio") | | Text | * [Word2vec](/wiki/Word2vec "Word2vec") * [Seq2seq](/wiki/Seq2seq "Seq2seq") * [GloVe](/wiki/GloVe "GloVe") * [BERT](/wiki/BERT_%28language_model%29 "BERT (language model)") * [T5](/wiki/T5_%28language_model%29 "T5 (language model)") * [Llama](/wiki/Llama_%28language_model%29 "Llama (language model)") * [Chinchilla AI](/wiki/Chinchilla_%28language_model%29 "Chinchilla (language model)") * [PaLM](/wiki/PaLM "PaLM") * [GPT](/wiki/Generative_pre-trained_transformer "Generative pre-trained transformer")   + [1](/wiki/GPT-1 "GPT-1")   + [2](/wiki/GPT-2 "GPT-2")   + [3](/wiki/GPT-3 "GPT-3")   + [J](/wiki/GPT-J "GPT-J")   + [ChatGPT](/wiki/ChatGPT "ChatGPT")   + [4](/wiki/GPT-4 "GPT-4")   + 4o   + [o1](/wiki/OpenAI_o1 "OpenAI o1")   + [o3](/wiki/OpenAI_o3 "OpenAI o3")   + [4.5](/wiki/GPT-4.5 "GPT-4.5")   + [4.1](/wiki/GPT-4.1 "GPT-4.1")   + [o4-mini](/wiki/OpenAI_o4-mini "OpenAI o4-mini") * [Claude](/wiki/Claude_%28language_model%29 "Claude (language model)") * [Gemini](/wiki/Gemini_%28language_model%29 "Gemini (language model)")   + [chatbot](/wiki/Gemini_%28chatbot%29 "Gemini (chatbot)") * [Grok](/wiki/Grok_%28chatbot%29 "Grok (chatbot)") * [LaMDA](/wiki/LaMDA "LaMDA") * [BLOOM](/wiki/BLOOM_%28language_model%29 "BLOOM (language model)") * [DBRX](/wiki/DBRX "DBRX") * [Project Debater](/wiki/Project_Debater "Project Debater") * [IBM Watson](/wiki/IBM_Watson "IBM Watson") * [IBM Watsonx](/wiki/IBM_Watsonx "IBM Watsonx") * [Granite](/wiki/IBM_Granite "IBM Granite") * [PanGu-Σ](/wiki/Huawei_PanGu "Huawei PanGu") * [DeepSeek](/wiki/DeepSeek_%28chatbot%29 "DeepSeek (chatbot)") * [Qwen](/wiki/Qwen "Qwen") | | Decisional | * [AlphaGo](/wiki/AlphaGo "AlphaGo") * [AlphaZero](/wiki/AlphaZero "AlphaZero") * [OpenAI Five](/wiki/OpenAI_Five "OpenAI Five") * [Self-driving car](/wiki/Self-driving_car "Self-driving car") * [MuZero](/wiki/MuZero "MuZero") * [Action selection](/wiki/Action_selection "Action selection")   + [AutoGPT](/wiki/AutoGPT "AutoGPT") * [Robot control](/wiki/Robot_control "Robot control") | |
| People | * [Alan Turing](/wiki/Alan_Turing "Alan Turing") * [Warren Sturgis McCulloch](/wiki/Warren_Sturgis_McCulloch "Warren Sturgis McCulloch") * [Walter Pitts](/wiki/Walter_Pitts "Walter Pitts") * [John von Neumann](/wiki/John_von_Neumann "John von Neumann") * [Claude Shannon](/wiki/Claude_Shannon "Claude Shannon") * [Shun'ichi Amari](/wiki/Shun%27ichi_Amari "Shun'ichi Amari") * [Kunihiko Fukushima](/wiki/Kunihiko_Fukushima "Kunihiko Fukushima") * [Takeo Kanade](/wiki/Takeo_Kanade "Takeo Kanade") * [Marvin Minsky](/wiki/Marvin_Minsky "Marvin Minsky") * [John McCarthy](/wiki/John_McCarthy_%28computer_scientist%29 "John McCarthy (computer scientist)") * [Nathaniel Rochester](/wiki/Nathaniel_Rochester_%28computer_scientist%29 "Nathaniel Rochester (computer scientist)") * [Allen Newell](/wiki/Allen_Newell "Allen Newell") * [Cliff Shaw](/wiki/Cliff_Shaw "Cliff Shaw") * [Herbert A. Simon](/wiki/Herbert_A._Simon "Herbert A. Simon") * [Oliver Selfridge](/wiki/Oliver_Selfridge "Oliver Selfridge") * [Frank Rosenblatt](/wiki/Frank_Rosenblatt "Frank Rosenblatt") * [Bernard Widrow](/wiki/Bernard_Widrow "Bernard Widrow") * [Joseph Weizenbaum](/wiki/Joseph_Weizenbaum "Joseph Weizenbaum") * [Seymour Papert](/wiki/Seymour_Papert "Seymour Papert") * [Seppo Linnainmaa](/wiki/Seppo_Linnainmaa "Seppo Linnainmaa") * [Paul Werbos](/wiki/Paul_Werbos "Paul Werbos") * [Geoffrey Hinton](/wiki/Geoffrey_Hinton "Geoffrey Hinton") * [John Hopfield](/wiki/John_Hopfield "John Hopfield") * [Jürgen Schmidhuber](/wiki/J%C3%BCrgen_Schmidhuber "Jürgen Schmidhuber") * [Yann LeCun](/wiki/Yann_LeCun "Yann LeCun") * [Yoshua Bengio](/wiki/Yoshua_Bengio "Yoshua Bengio") * [Lotfi A. Zadeh](/wiki/Lotfi_A._Zadeh "Lotfi A. Zadeh") * [Stephen Grossberg](/wiki/Stephen_Grossberg "Stephen Grossberg") * [Alex Graves](/wiki/Alex_Graves_%28computer_scientist%29 "Alex Graves (computer scientist)") * [James Goodnight](/wiki/James_Goodnight "James Goodnight") * [Andrew Ng](/wiki/Andrew_Ng "Andrew Ng") * [Fei-Fei Li](/wiki/Fei-Fei_Li "Fei-Fei Li") * [Ilya Sutskever](/wiki/Ilya_Sutskever "Ilya Sutskever") * [Alex Krizhevsky](/wiki/Alex_Krizhevsky "Alex Krizhevsky") * [Ian Goodfellow](/wiki/Ian_Goodfellow "Ian Goodfellow") * [Demis Hassabis](/wiki/Demis_Hassabis "Demis Hassabis") * [David Silver](/wiki/David_Silver_%28computer_scientist%29 "David Silver (computer scientist)") * [Andrej Karpathy](/wiki/Andrej_Karpathy "Andrej Karpathy") * [Ashish Vaswani](/wiki/Ashish_Vaswani "Ashish Vaswani") * [Noam Shazeer](/wiki/Noam_Shazeer "Noam Shazeer") * [Aidan Gomez](/wiki/Aidan_Gomez "Aidan Gomez") * [François Chollet](/wiki/Fran%C3%A7ois_Chollet "François Chollet") |
| Architectures | * [Neural Turing machine](/wiki/Neural_Turing_machine "Neural Turing machine") * [Differentiable neural computer](/wiki/Differentiable_neural_computer "Differentiable neural computer") * [Transformer](/wiki/Transformer_%28deep_learning_architecture%29 "Transformer (deep learning architecture)")   + [Vision transformer (ViT)](/wiki/Vision_transformer "Vision transformer") * [Recurrent neural network (RNN)](/wiki/Recurrent_neural_network "Recurrent neural network") * [Long short-term memory (LSTM)](/wiki/Long_short-term_memory "Long short-term memory") * [Gated recurrent unit (GRU)](/wiki/Gated_recurrent_unit "Gated recurrent unit") * [Echo state network](/wiki/Echo_state_network "Echo state network") * [Multilayer perceptron (MLP)](/wiki/Multilayer_perceptron "Multilayer perceptron") * [Convolutional neural network (CNN)](/wiki/Convolutional_neural_network "Convolutional neural network") * [Residual neural network (RNN)](/wiki/Residual_neural_network "Residual neural network") * [Highway network](/wiki/Highway_network "Highway network") * [Mamba](/wiki/Mamba_%28deep_learning_architecture%29 "Mamba (deep learning architecture)") * [Autoencoder](/wiki/Autoencoder "Autoencoder") * [Variational autoencoder (VAE)](/wiki/Variational_autoencoder "Variational autoencoder") * [Generative adversarial network (GAN)](/wiki/Generative_adversarial_network "Generative adversarial network") * [Graph neural network (GNN)](/wiki/Graph_neural_network "Graph neural network") |
| * [![](//upload.wikimedia.org/wikipedia/en/thumb/e/e2/Symbol_portal_class.svg/20px-Symbol_portal_class.svg.png)](/wiki/File%3ASymbol_portal_class.svg "Portal") Portals   + [Technology](/wiki/Portal%3ATechnology "Portal:Technology") * ![](//upload.wikimedia.org/wikipedia/en/thumb/9/96/Symbol_category_class.svg/20px-Symbol_category_class.svg.png) [Category](/wiki/Category%3AArtificial_intelligence "Category:Artificial intelligence")   + [Artificial neural networks](/wiki/Category%3AArtificial_neural_networks "Category:Artificial neural networks")   + [Machine learning](/wiki/Category%3AMachine_learning "Category:Machine learning") * ![](//upload.wikimedia.org/wikipedia/en/thumb/d/db/Symbol_list_class.svg/20px-Symbol_list_class.svg.png) List   + [Companies](/wiki/List_of_artificial_intelligence_companies "List of artificial intelligence companies")   + [Projects](/wiki/List_of_artificial_intelligence_projects "List of artificial intelligence projects") | |

| * [v](/wiki/Template%3AGenerative_AI "Template:Generative AI") * [t](/wiki/Template_talk%3AGenerative_AI "Template talk:Generative AI") * [e](/wiki/Special%3AEditPage/Template%3AGenerative_AI "Special:EditPage/Template:Generative AI")  [Generative AI](/wiki/Generative_artificial_intelligence "Generative artificial intelligence") | |
| --- | --- |
| Concepts | * [Autoencoder](/wiki/Autoencoder "Autoencoder") * [Deep learning](/wiki/Deep_learning "Deep learning") * [Fine-tuning](/wiki/Fine-tuning_%28deep_learning%29 "Fine-tuning (deep learning)") * [Foundation model](/wiki/Foundation_model "Foundation model") * [Generative adversarial network](/wiki/Generative_adversarial_network "Generative adversarial network") * [Generative pre-trained transformer](/wiki/Generative_pre-trained_transformer "Generative pre-trained transformer") * [Large language model](/wiki/Large_language_model "Large language model") * [Model Context Protocol](/wiki/Model_Context_Protocol "Model Context Protocol") * [Neural network](/wiki/Neural_network_%28machine_learning%29 "Neural network (machine learning)") * [Prompt engineering](/wiki/Prompt_engineering "Prompt engineering") * [Reinforcement learning from human feedback](/wiki/Reinforcement_learning_from_human_feedback "Reinforcement learning from human feedback") * [Retrieval-augmented generation](/wiki/Retrieval-augmented_generation "Retrieval-augmented generation") * [Self-supervised learning](/wiki/Self-supervised_learning "Self-supervised learning") * [Stochastic parrot](/wiki/Stochastic_parrot "Stochastic parrot") * [Synthetic data](/wiki/Synthetic_data "Synthetic data") * [Top-p sampling](/wiki/Top-p_sampling "Top-p sampling") * [Transformer](/wiki/Transformer_%28deep_learning_architecture%29 "Transformer (deep learning architecture)") * [Variational autoencoder](/wiki/Variational_autoencoder "Variational autoencoder") * [Vibe coding](/wiki/Vibe_coding "Vibe coding") * [Vision transformer](/wiki/Vision_transformer "Vision transformer") * [Waluigi effect](/wiki/Waluigi_effect "Waluigi effect") * [Word embedding](/wiki/Word_embedding "Word embedding") |
| Models | |  |  | | --- | --- | | Text | * [Character.ai](/wiki/Character.ai "Character.ai") * [ChatGLM](/wiki/Zhipu_AI#ChatGLM "Zhipu AI") * [Claude](/wiki/Claude_%28language_model%29 "Claude (language model)") * [DeepSeek](/wiki/DeepSeek_%28chatbot%29 "DeepSeek (chatbot)") * [Ernie](/wiki/Ernie_Bot "Ernie Bot") * [Gemini](/wiki/Gemini_%28chatbot%29 "Gemini (chatbot)") * [GPT](/wiki/Generative_pre-trained_transformer "Generative pre-trained transformer")   + [1](/wiki/GPT-1 "GPT-1")   + [2](/wiki/GPT-2 "GPT-2")   + [3](/wiki/GPT-3 "GPT-3")   + [J](/wiki/GPT-J "GPT-J")   + [ChatGPT](/wiki/ChatGPT "ChatGPT")   + [4](/wiki/GPT-4 "GPT-4")   + 4o   + [o1](/wiki/OpenAI_o1 "OpenAI o1")   + [o3](/wiki/OpenAI_o3 "OpenAI o3")   + [4.5](/wiki/GPT-4.5 "GPT-4.5")   + [4.1](/wiki/GPT-4.1 "GPT-4.1")   + [o4-mini](/wiki/OpenAI_o4-mini "OpenAI o4-mini") * [Grok](/wiki/Grok_%28chatbot%29 "Grok (chatbot)") * [Hunyuan Turbo S](/wiki/Tencent#Research "Tencent") * [Kimi](/wiki/Moonshot_AI#Kimi "Moonshot AI") * [Llama](/wiki/Llama_%28language_model%29 "Llama (language model)") * [Microsoft Copilot](/wiki/Microsoft_Copilot "Microsoft Copilot") * [MiniMax-M1](/wiki/MiniMax_%28company%29#Technology "MiniMax (company)") * [Magistral Medium](/wiki/Mistral_AI#Magistral_Small_and_Magistral_Medium "Mistral AI") * [Qwen](/wiki/Qwen "Qwen") * [Yi-Lightning](/wiki/01.AI#Yi "01.AI") | | Coding | * [Claude Code](/wiki/Claude_%28language_model%29 "Claude (language model)") * [Cursor](/wiki/Cursor_%28code_editor%29 "Cursor (code editor)") * [Devstral](/wiki/Mistral_AI "Mistral AI") * [GitHub Copilot](/wiki/GitHub_Copilot "GitHub Copilot") * [Granite Code](/wiki/IBM_Granite#Foundation_models "IBM Granite") * [Kimi-Dev](/wiki/Moonshot_AI "Moonshot AI") * [Replit](/wiki/Replit "Replit") * [Xcode](/wiki/Xcode "Xcode") | | [Image](/wiki/Text-to-image_model "Text-to-image model") | * [Aurora](/wiki/Aurora_%28text-to-image_model%29 "Aurora (text-to-image model)") * [Firefly](/wiki/Adobe_Firefly "Adobe Firefly") * [Flux](/wiki/Flux_%28text-to-image_model%29 "Flux (text-to-image model)") * [GPT Image 1](#GPT_Image_1) * [Ideogram](/wiki/Ideogram_%28text-to-image_model%29 "Ideogram (text-to-image model)") * [Imagen](/wiki/Imagen_%28text-to-image_model%29 "Imagen (text-to-image model)") * [Leonardo](/wiki/Canva#Acquisitions "Canva") * [Midjourney](/wiki/Midjourney "Midjourney") * [Recraft](/wiki/Recraft "Recraft") * [Seedream](/wiki/ByteDance "ByteDance") * [Stable Diffusion](/wiki/Stable_Diffusion "Stable Diffusion") | | [Video](/wiki/Text-to-video_model "Text-to-video model") | * [Dream Machine](/wiki/Dream_Machine_%28text-to-video_model%29 "Dream Machine (text-to-video model)") * [Hailuo AI](/wiki/MiniMax_%28company%29#Hailuo_AI "MiniMax (company)") * [Kling](/wiki/Kling_%28text-to-video_model%29 "Kling (text-to-video model)") * [Midjourney Video](/wiki/Midjourney "Midjourney") * [Runway Gen](/wiki/Runway_%28company%29#Services_and_technologies "Runway (company)") * [Seedance](/wiki/ByteDance "ByteDance") * [Sora](/wiki/Sora_%28text-to-video_model%29 "Sora (text-to-video model)") * [Veo](/wiki/Veo_%28text-to-video_model%29 "Veo (text-to-video model)") * [Wan](/wiki/Alibaba_Group#Cloud_computing_and_artificial_intelligence_technology "Alibaba Group") | | [Speech](/wiki/Speech_synthesis#Text-to-speech_systems "Speech synthesis") | * [15.ai](/wiki/15.ai "15.ai") * [ElevenLabs](/wiki/ElevenLabs "ElevenLabs") * [Speech-02](/wiki/MiniMax_%28company%29#Technology "MiniMax (company)") * [WaveNet](/wiki/WaveNet "WaveNet") | | Music | * [Endel](/wiki/Endel_%28app%29 "Endel (app)") * [Lyria](/wiki/Google_DeepMind#Music_generation "Google DeepMind") * [Riffusion](/wiki/Riffusion "Riffusion") * [Suno AI](/wiki/Suno_AI "Suno AI") * [Udio](/wiki/Udio "Udio") | |
| [Agents](/wiki/Intelligent_agent "Intelligent agent") | * [AutoGLM](/wiki/Zhipu_AI#AutoGLM "Zhipu AI") * [AutoGPT](/wiki/AutoGPT "AutoGPT") * [Devin AI](/wiki/Devin_AI "Devin AI") * [Manus](/wiki/Manus_%28AI_agent%29 "Manus (AI agent)") * [OpenAI Codex](/wiki/OpenAI_Codex "OpenAI Codex") * [Operator](/wiki/OpenAI_Operator "OpenAI Operator") * [Replit Agent](/wiki/Replit "Replit") |
| [Companies](/wiki/List_of_artificial_intelligence_companies "List of artificial intelligence companies") | * [01.AI](/wiki/01.AI "01.AI") * [Aleph Alpha](/wiki/Aleph_Alpha "Aleph Alpha") * [Anthropic](/wiki/Anthropic "Anthropic") * [Baichuan](/wiki/Baichuan "Baichuan") * [Canva](/wiki/Canva "Canva") * [Cognition AI](/wiki/Cognition_AI "Cognition AI") * [Cohere](/wiki/Cohere "Cohere") * [Contextual AI](/wiki/Contextual_AI "Contextual AI") * [DeepSeek](/wiki/DeepSeek "DeepSeek") * [Google DeepMind](/wiki/Google_DeepMind "Google DeepMind") * [HeyGen](/wiki/HeyGen "HeyGen") * [Hugging Face](/wiki/Hugging_Face "Hugging Face") * [Krikey AI](/wiki/Krikey_AI "Krikey AI") * [Kuaishou](/wiki/Kuaishou "Kuaishou") * [Luma Labs](/wiki/Luma_Labs "Luma Labs") * [Meta AI](/wiki/Meta_AI "Meta AI") * [MiniMax](/wiki/MiniMax_%28company%29 "MiniMax (company)") * [Mistral AI](/wiki/Mistral_AI "Mistral AI") * [Moonshot AI](/wiki/Moonshot_AI "Moonshot AI") * [OpenAI](/wiki/OpenAI "OpenAI") * [Runway](/wiki/Runway_%28company%29 "Runway (company)") * [Safe Superintelligence](/wiki/Safe_Superintelligence_Inc. "Safe Superintelligence Inc.") * [Scale AI](/wiki/Scale_AI "Scale AI") * [Stability AI](/wiki/Stability_AI "Stability AI") * [Synthesia](/wiki/Synthesia_%28company%29 "Synthesia (company)") * [Thinking Machines Lab](/wiki/Thinking_Machines_Lab "Thinking Machines Lab") * [xAI](/wiki/XAI_%28company%29 "XAI (company)") * [Zhipu AI](/wiki/Zhipu_AI "Zhipu AI") |
| * ![](//upload.wikimedia.org/wikipedia/en/thumb/9/96/Symbol_category_class.svg/20px-Symbol_category_class.svg.png) **[Category](/wiki/Category%3AGenerative_artificial_intelligence "Category:Generative artificial intelligence")** * ![](//upload.wikimedia.org/wikipedia/en/thumb/4/4a/Commons-logo.svg/20px-Commons-logo.svg.png) **[Commons](https://commons.wikimedia.org/wiki/Category%3AGenerative_artificial_intelligence "commons:Category:Generative artificial intelligence")** | |

![](https://en.wikipedia.org/wiki/Special:CentralAutoLogin/start?type=1x1&usesul3=1)

Retrieved from "<https://en.wikipedia.org/w/index.php?title=GPT-4o&oldid=1300788525>"

### Build a Weather Tool

In [45]:
import requests

@tool
def get_weather(query: str) -> list:
    """Search weatherapi to get the current weather."""
    base_url = "http://api.weatherapi.com/v1/current.json"
    complete_url = f"{base_url}?key={os.getenv('WEATHER_API_KEY')}&q={query}"

    response = requests.get(complete_url)
    data = response.json()
    if data.get("location"):
        return data
    else:
        return "Weather Data Not Found"

In [46]:
get_weather.invoke("Bangalore")

{'location': {'name': 'Bangalore',
  'region': 'Karnataka',
  'country': 'India',
  'lat': 12.9833,
  'lon': 77.5833,
  'tz_id': 'Asia/Kolkata',
  'localtime_epoch': 1752706442,
  'localtime': '2025-07-17 04:24'},
 'current': {'last_updated_epoch': 1752705900,
  'last_updated': '2025-07-17 04:15',
  'temp_c': 21.2,
  'temp_f': 70.2,
  'is_day': 0,
  'condition': {'text': 'Partly cloudy',
   'icon': '//cdn.weatherapi.com/weather/64x64/night/116.png',
   'code': 1003},
  'wind_mph': 9.4,
  'wind_kph': 15.1,
  'wind_degree': 251,
  'wind_dir': 'WSW',
  'pressure_mb': 1011.0,
  'pressure_in': 29.85,
  'precip_mm': 0.38,
  'precip_in': 0.01,
  'humidity': 94,
  'cloud': 75,
  'feelslike_c': 21.2,
  'feelslike_f': 70.2,
  'windchill_c': 21.3,
  'windchill_f': 70.3,
  'heatindex_c': 24.2,
  'heatindex_f': 75.6,
  'dewpoint_c': 19.1,
  'dewpoint_f': 66.4,
  'vis_km': 6.0,
  'vis_miles': 3.0,
  'uv': 0.0,
  'gust_mph': 14.7,
  'gust_kph': 23.6}}

In [47]:
import rich

result = get_weather.invoke("Zurich")
rich.print_json(data=result)

{
  "location": {
    "name": "Zurich",
    "region": "",
    "country": "Switzerland",
    "lat": 47.3667,
    "lon": 8.55,
    "tz_id": "Europe/Zurich",
    "localtime_epoch": 1752706443,
    "localtime": "2025-07-17 00:54"
  },
  "current": {
    "last_updated_epoch": 1752705900,
    "last_updated": "2025-07-17 00:45",
    "temp_c": 15.2,
    "temp_f": 59.4,
    "is_day": 0,
    "condition": {
      "text": "Light rain",
      "icon": "//cdn.weatherapi.com/weather/64x64/night/296.png",
      "code": 1183
    },
    "wind_mph": 2.5,
    "wind_kph": 4.0,
    "wind_degree": 224,
    "wind_dir": "SW",
    "pressure_mb": 1019.0,
    "pressure_in": 30.09,
    "precip_mm": 0.11,
    "precip_in": 0.0,
    "humidity": 100,
    "cloud": 75,
    "feelslike_c": 15.2,
    "feelslike_f": 59.4,
    "windchill_c": 14.4,
    "windchill_f": 58.0,
    "heatindex_c": 13.9,
    "heatindex_f": 57.0,
    "dewpoint_c": 12.1,
    "dewpoint_f": 53.7,
    "vis_km": 10.0,
    "vis_miles": 6.0,
    "uv": 0.0,
    "gust_mph": 4.5,
    "gust_kph": 7.2
  }
}

## Explore LLM tool calling with custom tools

An agent is basically an LLM which has the capability to automatically call relevant functions to perform complex or tool-based tasks based on input human prompts.

Tool calling also popularly known as function calling is the ability to reliably enable such LLMs to call external tools and APIs.

We will leverate the custom tools we created earlier in the previous section and try to see if the LLM can automatically call the right tools based on input prompts

### Tool calling for LLMs with native support for tool or function calling

Tool calling allows a model to respond to a given prompt by generating output that matches a user-defined schema. While the name implies that the model is performing some action, this is actually not the case! The model is coming up with the arguments to a tool, and actually running the tool (or not) is up to the user or agent defined by the user.

Many LLM providers, including Anthropic, Cohere, Google, Mistral, OpenAI, and others, support variants of a tool calling feature. These features typically allow requests to the LLM to include available tools and their schemas, and for responses to include calls to these tools.



In [48]:
from langchain_openai import ChatOpenAI

chatgpt = ChatOpenAI(model="gpt-4o", temperature=0)

In [49]:
tools = [multiply, search_web_extract_info, get_weather]
chatgpt_with_tools = chatgpt.bind_tools(tools)

In [62]:
tools

[StructuredTool(name='multiply', description='use to multiply numbers', args_schema=<class '__main__.CalculatorInput'>, return_direct=True, func=<function multiply at 0x0000023AA9BC62A0>),
 StructuredTool(name='search_web_extract_info', description='Search the web for a query and extracts useful information from the search links', args_schema=<class 'langchain_core.utils.pydantic.search_web_extract_info'>, func=<function search_web_extract_info at 0x0000023AADF698A0>),
 StructuredTool(name='get_weather', description='Search weatherapi to get the current weather.', args_schema=<class 'langchain_core.utils.pydantic.get_weather'>, func=<function get_weather at 0x0000023AADF6A0C0>)]

In [82]:
# LLMs are still not perfect in tool calling so you might need to play around with the following prompt
prompt = """
            Given only the tools at your disposal, mention tool calls for the following tasks:
            Do not change the query given for any search tasks
            1. What is 2.1 times 3.5
            2. What is the current weather in Greenland today
            3. What are the 4 major Agentic AI Design Patterns
         """

results = chatgpt_with_tools.invoke(prompt)

In [83]:
results

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_m3WfEKpqDuoTFAZBcRnUDW5y', 'function': {'arguments': '{"a": 2.1, "b": 3.5}', 'name': 'multiply'}, 'type': 'function'}, {'id': 'call_jiFHA3rwA3Mm9Wrh3T1WnJJx', 'function': {'arguments': '{"query": "Greenland"}', 'name': 'get_weather'}, 'type': 'function'}, {'id': 'call_9mRLqvLtNHGXfIONgyF1DWy2', 'function': {'arguments': '{"query": "4 major Agentic AI Design Patterns"}', 'name': 'search_web_extract_info'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 74, 'prompt_tokens': 182, 'total_tokens': 256, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--96bbf118-186b-4e7a-8b0a-bd6512199eab-0', tool_calls

In [84]:
results.content

''

In [85]:
results.tool_calls

[{'name': 'multiply',
  'args': {'a': 2.1, 'b': 3.5},
  'id': 'call_m3WfEKpqDuoTFAZBcRnUDW5y',
  'type': 'tool_call'},
 {'name': 'get_weather',
  'args': {'query': 'Greenland'},
  'id': 'call_jiFHA3rwA3Mm9Wrh3T1WnJJx',
  'type': 'tool_call'},
 {'name': 'search_web_extract_info',
  'args': {'query': '4 major Agentic AI Design Patterns'},
  'id': 'call_9mRLqvLtNHGXfIONgyF1DWy2',
  'type': 'tool_call'}]

In [86]:
multiply

StructuredTool(name='multiply', description='use to multiply numbers', args_schema=<class '__main__.CalculatorInput'>, return_direct=True, func=<function multiply at 0x0000023AA9BC62A0>)

In [97]:
for tool_call in results.tool_calls:
    print(tool_call["name"])


multiply
get_weather
search_web_extract_info


In [99]:
toolkit["multiply"].name

'multiply'

In [100]:
toolkit = {
    "multiply": multiply,
    "search_web_extract_info": search_web_extract_info,
    "get_weather": get_weather
}

for tool_call in results.tool_calls:
    selected_tool = toolkit[tool_call["name"].lower()]
    print(f"Calling tool: {selected_tool.name}")
    tool_output = selected_tool.invoke(tool_call["args"])
    print(tool_output)
    print()

Calling tool: multiply
7.3500000000000005

Calling tool: get_weather
{'location': {'name': 'Nuuk', 'region': 'Vestgronland', 'country': 'Greenland', 'lat': 64.183, 'lon': -51.75, 'tz_id': 'America/Nuuk', 'localtime_epoch': 1752707099, 'localtime': '2025-07-16 22:04'}, 'current': {'last_updated_epoch': 1752706800, 'last_updated': '2025-07-16 22:00', 'temp_c': 13.0, 'temp_f': 55.4, 'is_day': 0, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/night/116.png', 'code': 1003}, 'wind_mph': 4.7, 'wind_kph': 7.6, 'wind_degree': 43, 'wind_dir': 'NE', 'pressure_mb': 1007.0, 'pressure_in': 29.74, 'precip_mm': 0.03, 'precip_in': 0.0, 'humidity': 67, 'cloud': 75, 'feelslike_c': 12.6, 'feelslike_f': 54.7, 'windchill_c': 7.0, 'windchill_f': 44.7, 'heatindex_c': 8.2, 'heatindex_f': 46.8, 'dewpoint_c': 5.6, 'dewpoint_f': 42.1, 'vis_km': 10.0, 'vis_miles': 6.0, 'uv': 0.0, 'gust_mph': 9.9, 'gust_kph': 15.9}}

Calling tool: search_web_extract_info


100%|██████████| 5/5 [00:03<00:00,  1.28it/s]

['Top 4 Agentic AI Design Patterns\n[Master Generative AI with 10+ Real-world Projects in 2025!\n\n* ######\n\n  d\n:* ######\n\n  h\n:* ######\n\n  m\n:* ######\n\n  s\n\nDownload Projects](https://www.analyticsvidhya.com/pinnacleplus/pinnacleplus-projects?utm_source=blog_india&utm_medium=desktop_flashstrip&utm_campaign=15-Feb-2025||&utm_content=projects)\n\n[![Analytics Vidhya](https://www.analyticsvidhya.com/wp-content/themes/analytics-vidhya/icon/av-logo-svg.svg)](https://www.analyticsvidhya.com/blog/)\n\n* [Free Courses](https://www.analyticsvidhya.com/courses/?ref=Navbar)\n* [Learning Paths](https://www.analyticsvidhya.com/learning-path/chat/?utm_source=blog&utm_medium=navbar)\n* [GenAI Pinnacle Plus](https://www.analyticsvidhya.com/pinnacleplus/?ref=blognavbar)\n  New\n* [Agentic AI Pioneer](https://www.analyticsvidhya.com/agenticaipioneer/?ref=blognavbar)\n* [DHS 2025](https://www.analyticsvidhya.com/datahacksummit?utm_source=blog&utm_medium=navbar)\n\n* Login\n\n  + Switch Mod

### Tool calling for LLMs without native support for tool or function calling

Some models like ChatGPT have been fine-tuned for tool calling and provide a dedicated API for tool calling. Generally, such models are better at tool calling than non-fine-tuned models, and are recommended for use cases that require tool calling.

Here we will explore an alternative method to invoke tools if you're using a model that does not natively support tool calling (even though we use ChatGPT here which supports it, we will assume it could be any LLM which doesn't support tool calling).

We'll do this by simply writing a prompt that will get the model to invoke the appropriate tools.

In [88]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.tools import render_text_description

rendered_tools = render_text_description(tools)
print(rendered_tools)

multiply(a: float, b: float) -> float - use to multiply numbers
search_web_extract_info(query: str) -> list - Search the web for a query and extracts useful information from the search links
get_weather(query: str) -> list - Search weatherapi to get the current weather.


In [89]:
system_prompt = f"""\
You are an assistant that has access to the following set of tools.
Here are the names and descriptions for each tool:

{rendered_tools}

Given the user instructions, for each instruction do the following:
 - Return the name and input of the tool to use.
 - Return your response as a JSON blob with 'name' and 'arguments' keys.
 - The `arguments` should be a dictionary, with keys corresponding
   to the argument names and the values corresponding to the requested values.
 - IMPORTANT: Your response must be valid JSON format only, no additional text.
"""

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("user", "{input}")
    ]
)

In [90]:
instructions = [
                  {"input" : "What is 2.1 times 3.5"},
                  {"input" : "What is the current weather in Greenland"},
                  {"input" : "Tell me about the current state of Agentic AI in the industry" }
               ]

In [91]:
from langchain_core.output_parsers import JsonOutputParser

chain = (prompt
            |
         chatgpt
            |
         JsonOutputParser())

In [92]:
responses = chain.map().invoke(instructions)

In [93]:
responses

[{'name': 'multiply', 'arguments': {'a': 2.1, 'b': 3.5}},
 {'name': 'get_weather', 'arguments': {'query': 'Greenland'}},
 {'name': 'search_web_extract_info',
  'arguments': {'query': 'current state of Agentic AI in the industry 2023'}}]

In [94]:
toolkit = {
    "multiply": multiply,
    "search_web_extract_info": search_web_extract_info,
    "get_weather": get_weather
}

for tool_call in responses:
    selected_tool = toolkit[tool_call["name"].lower()]
    print(f"Calling tool: {tool_call['name']}")
    tool_output = selected_tool.invoke(tool_call["arguments"])
    print(tool_output)
    print()

Calling tool: multiply
7.3500000000000005

Calling tool: get_weather
{'location': {'name': 'Nuuk', 'region': 'Vestgronland', 'country': 'Greenland', 'lat': 64.183, 'lon': -51.75, 'tz_id': 'America/Nuuk', 'localtime_epoch': 1752706531, 'localtime': '2025-07-16 21:55'}, 'current': {'last_updated_epoch': 1752705900, 'last_updated': '2025-07-16 21:45', 'temp_c': 13.2, 'temp_f': 55.8, 'is_day': 0, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/night/116.png', 'code': 1003}, 'wind_mph': 7.2, 'wind_kph': 11.5, 'wind_degree': 10, 'wind_dir': 'N', 'pressure_mb': 1007.0, 'pressure_in': 29.74, 'precip_mm': 0.02, 'precip_in': 0.0, 'humidity': 67, 'cloud': 75, 'feelslike_c': 12.3, 'feelslike_f': 54.1, 'windchill_c': 5.5, 'windchill_f': 41.8, 'heatindex_c': 7.6, 'heatindex_f': 45.6, 'dewpoint_c': 5.2, 'dewpoint_f': 41.4, 'vis_km': 10.0, 'vis_miles': 6.0, 'uv': 0.1, 'gust_mph': 15.0, 'gust_kph': 24.2}}

Calling tool: search_web_extract_info


100%|██████████| 5/5 [00:06<00:00,  1.22s/it]

['The Rise of Agentic AI: What Every Technology Leader Needs to Know\nAgree & Join LinkedIn\n\nBy clicking Continue to join or sign in, you agree to LinkedIn’s [User Agreement](/legal/user-agreement?trk=linkedin-tc_auth-button_user-agreement), [Privacy Policy](/legal/privacy-policy?trk=linkedin-tc_auth-button_privacy-policy), and [Cookie Policy](/legal/cookie-policy?trk=linkedin-tc_auth-button_cookie-policy).\n\n[Skip to main content](#main-content)\n\n[LinkedIn](/?trk=article-ssr-frontend-pulse_nav-header-logo)\n\n* [Articles](https://www.linkedin.com/pulse/topics/home/?trk=article-ssr-frontend-pulse_guest_nav_menu_articles)\n* [People](https://www.linkedin.com/pub/dir/%2B/%2B?trk=article-ssr-frontend-pulse_guest_nav_menu_people)\n* [Learning](https://www.linkedin.com/learning/search?trk=article-ssr-frontend-pulse_guest_nav_menu_learning)\n* [Jobs](https://www.linkedin.com/jobs/search?trk=article-ssr-frontend-pulse_guest_nav_menu_jobs)\n* [Games](https://www.linkedin.com/games?trk=art

In [95]:
for doc in tool_output:
    print(doc)
    print()

The Rise of Agentic AI: What Every Technology Leader Needs to Know
Agree & Join LinkedIn

By clicking Continue to join or sign in, you agree to LinkedIn’s [User Agreement](/legal/user-agreement?trk=linkedin-tc_auth-button_user-agreement), [Privacy Policy](/legal/privacy-policy?trk=linkedin-tc_auth-button_privacy-policy), and [Cookie Policy](/legal/cookie-policy?trk=linkedin-tc_auth-button_cookie-policy).

[Skip to main content](#main-content)

[LinkedIn](/?trk=article-ssr-frontend-pulse_nav-header-logo)

* [Articles](https://www.linkedin.com/pulse/topics/home/?trk=article-ssr-frontend-pulse_guest_nav_menu_articles)
* [People](https://www.linkedin.com/pub/dir/%2B/%2B?trk=article-ssr-frontend-pulse_guest_nav_menu_people)
* [Learning](https://www.linkedin.com/learning/search?trk=article-ssr-frontend-pulse_guest_nav_menu_learning)
* [Jobs](https://www.linkedin.com/jobs/search?trk=article-ssr-frontend-pulse_guest_nav_menu_jobs)
* [Games](https://www.linkedin.com/games?trk=article-ssr-fronte